# Tutorial - Overview of TicTs

---

## Design

TimeSeries object is inheriting from [Sorted Dict](http://www.grantjenks.com/docs/sortedcontainers/sorteddict.html) of *sortedcontainers* python library to ensure good performances and ease of use.

- designed to **feel like** manipulating a **pandas.Series** in the unevenly-spaced timeseries world.
- time intervals are considered **left closed** and **right open**.
- ensure timezone localization.

Let's see the potential !

In [1]:
# Plotting imports
from bokeh.plotting import figure, show
from bokeh.layouts import row
from bokeh.io import output_notebook
output_notebook()

# Standard lib
from copy import deepcopy

from ticts import TimeSeries

Loading BokehJS ...

In [98]:
# Plot tools with bokeh
import arrow
X_RANGE = (arrow.get('2018-12-31 23:57:00').timestamp * 1000, arrow.get('2019-01-01 00:20:00').timestamp * 1000)
Y_RANGE = (-5, 30)
def get_figure(ts, title='Unevenly Spaced', x_range=X_RANGE, y_range=Y_RANGE):
    p = figure(title=title, x_axis_type="datetime",
               x_axis_label="timestamp", y_axis_label="IoT sensor",
               x_range=x_range, y_range=y_range,
               plot_width=400, plot_height=250)
    p.step(ts.keys(), ts.values(), line_width=2, line_dash="4 4", mode="after")
    p.circle(ts.keys(), ts.values(), fill_color="red", size=6)
    return p

def plot(ts):
    show(get_figure(ts))
    
def plot_row(ts1, ts2, *args, x_range=X_RANGE, y_range=Y_RANGE):
    figures = []
    for i, ts in enumerate([ts1, ts2, *args]):
        figure = get_figure(ts, title='ts{}'.format(i+1), x_range=x_range, y_range=y_range)
        figures.append(figure)
    show(row(figures))

# Instanciation

In [99]:
dct = {
  '2019-01-01': 10,
  '2019-01-01 00:10:00': 20,
  '2019-01-01 00:15:00': 10,
}
ts = TimeSeries(dct)

# Can set item like a dict:
ts['2019-01-01 00:17:00'] = 1

In [100]:
# Let's see it:
print(ts)

<TimeSeries>
2019-01-01T00:00:00+00:00: 10,
2019-01-01T00:10:00+00:00: 20,
2019-01-01T00:15:00+00:00: 10,
2019-01-01T00:17:00+00:00: 1,


In [101]:
plot(ts)

# Some rules

- A value is applied from its timestamp, until the next one

In [102]:
# Can access any timestamp, if the timestamp is not part of the keys, it will get the previous value
dt = '2019-01-01 00:13:00'
value = ts[dt]
print("At '{}' got {}".format(dt, value))

At '2019-01-01 00:13:00' got 20


In [103]:
# Above right bound
dt = '2060-12-30'
value = ts[dt]
print("At '{}' got {}".format(dt, value))

At '2060-12-30' got 1


In [104]:
# Below left bound
dt = '2018-12-30'
value = ts[dt]
print("At '{}' got {}".format(dt, value))

At '2018-12-30' got None


In [105]:
# This happen when no default is set, as there is no previous value !
# Here is how to set your default:
ts_withdefault = TimeSeries(ts, default=100)
value = ts_withdefault[dt]
print("At '{}' got {}".format(dt, value))

# or
ts_withdefault.default=5
value = ts_withdefault[dt]
print("At '{}' got {}".format(dt, value))

At '2018-12-30' got 100
At '2018-12-30' got 5


## Operations

- sum and rsum
- sub
- mul
- div
- le
- lt
- ge
- gt
- ceil
- floor

In [106]:
# Let's define another one
ts2 = TimeSeries(default=0)
ts2['2019-01-01 00:07:00'] = -1
ts2['2019-01-01 00:10:00'] = 5
ts2['2019-01-01 00:12:00'] = 14

In [109]:
# Addition
plot_row(ts, ts + 10, y_range=(-5, 40))
plot_row(ts, ts2, ts + ts2, y_range=(-5, 40))

In [108]:
# Substraction
plot_row(ts, ts - 10)
plot_row(ts, ts2, ts - ts2)

In [82]:
# Floor (it applies a max on each key)
plot_row(ts, ts.floor(10))
plot_row(ts, ts2, ts.floor(ts2))

In [85]:
# Ceil (it applies a min on each key)
plot_row(ts, ts.ceil(10))
plot_row(ts, ts2, ts.ceil(ts2))

## Some sugar

In [84]:
# Go back to Evenly Spaced timeseries
from datetime import timedelta
onemin = timedelta(minutes=1)
evenly_ts = ts.sample(freq=onemin)
plot(evenly_ts)

In [67]:
# Compact
plot(evenly_ts.compact())

In [68]:
# if you got pandas installed
ts.to_dataframe()

,value
2019-01-01 00:00:00+00:00,10
2019-01-01 00:10:00+00:00,20
2019-01-01 00:15:00+00:00,10
2019-01-01 00:17:00+00:00,1
